# Caderno 7 - Usa GPT e Llama para reescrever o enunciado

## 1. Chaves de acesso e outras variáveis

In [1]:
from getpass import getpass

GROQ_API = getpass("API Groq")
OPENAI_API = getpass("API OpenAI")

MODELO_LLAMA = "llama3-70b-8192"
MODELO_GPT = "gpt-3.5-turbo-0125"

API Groq ········
API OpenAI ········


In [2]:
PASTA_DADOS = './dados/'
PASTA_RESULTADO_CADERNO = f'{PASTA_DADOS}outputs/7_reescreve_enunciado_gpt_llama/'

NOME_ARQUIVO_RESULTADO_LLAMA = f'{PASTA_RESULTADO_CADERNO}enunciados_reescritos_llama.pickle'
NOME_ARQUIVO_RESULTADO_GPT = f'{PASTA_RESULTADO_CADERNO}enunciados_reescritos_gpt.pickle'

## 2. Carrega os documentos

In [3]:
import pandas as pd

# A pasta dos JURIS aqui não é a pasta original, e sim o resultado do caderno 1 (os documentos já estão filtrados)
PASTA_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'

# Carrega os arquivos 
def carrega_juris_tcu():
    doc1 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_1.csv', sep='|')
    doc2 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_2.csv', sep='|')
    doc3 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_3.csv', sep='|')
    doc4 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_4.csv', sep='|')
    doc = pd.concat([doc1, doc2, doc3, doc4], ignore_index=True)
    query = pd.read_csv(f'{PASTA_JURIS_TCU}query_tratado.csv', sep='|')
    qrel = pd.read_csv(f'{PASTA_JURIS_TCU}qrel_tratado.csv', sep='|')

    return doc, query, qrel

docs, _, _ = carrega_juris_tcu()

## 3. Define o prompt de sistema que será usado no GPT e no Llama

In [4]:
system_message = """Você é um especialista em sistemas de busca que usam o algoritmo BM25 e está trabalhando na indexação de uma base de dados de jurisprudência do Tribunal de Contas da União. Essa base está sofrendo com o problema de descasamento de vocabulário, ou seja, o usuário usa termos de pesquisa que não estão no enunciado da jurisprudência. Trata-se de um problema comum, pois o usuário não sabe como o enunciado está escrito.

Para mitigar esse problema, além do enunciado original, será indexada uma versão reescrita do enunciado usando sinônimos. Dessa forma, espera-se que o usuário da pesquisa tenha maior probabilidade de encontrar o que procura.

Sua tarefa é reescrever o enunciado. Procure sinônimos mais comuns para as palavras usadas no enunciado original. O público alvo da pesquisa é o cidadão em geral, com variados graus de instrução. Por isso, o enunciado deve ser reescrito de forma simplificada.

Tudo o que você responder será indexado. Por isso, não forneça nada além da reescrita do enunciado.
"""

# Esse formato de mensagens é usado tanto na API da OpenAI quanto na API do Groq
def get_messages(enunciado):
    return [
        { "role": "system", "content": system_message },
        { "role": "user", "content": enunciado}
    ]

# 4. Define chamadas para GPT e Llama

In [5]:
from groq import Groq
from openai import OpenAI

clientGroq = Groq(api_key=GROQ_API)
clientOpenAI = OpenAI(api_key=OPENAI_API)

def completa_llama(enunciado):
    response = clientGroq.chat.completions.create(
        model=MODELO_LLAMA,
        messages=get_messages(enunciado),
        temperature=0,
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
    )    
    return response.choices[0].message.content

def completa_gpt(enunciado):
    response = clientOpenAI.chat.completions.create(
        model=MODELO_GPT,
        messages=get_messages(enunciado),
        temperature=1,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [6]:
%%time
# Testa com um enunciado qualquer pra ver se as chamadas estão funcionando:
enunciado = "A invalidez permanente é incompatível com o exercício de qualquer cargo público, razão pela qual é indevida a acumulação de proventos de invalidez permanente com remuneração decorrente do exercício de outro cargo, cabendo restituição ao erário dos proventos recebidos durante a acumulação ilegal."
print(enunciado)
print('.'*100)
print(completa_llama(enunciado))
print('.'*100)
print(completa_gpt(enunciado))

A invalidez permanente é incompatível com o exercício de qualquer cargo público, razão pela qual é indevida a acumulação de proventos de invalidez permanente com remuneração decorrente do exercício de outro cargo, cabendo restituição ao erário dos proventos recebidos durante a acumulação ilegal.
....................................................................................................
A incapacidade permanente impede o exercício de qualquer função pública, portanto, é injustificável receber ao mesmo tempo benefícios por incapacidade permanente e salário de outro cargo, devendo ser devolvidos ao Estado os valores recebidos durante a acumulação irregular.
....................................................................................................
A incapacidade permanente não permite trabalhar em cargos públicos, por isso não é correto acumular a aposentadoria por invalidez com o salário de outro cargo. Os valores recebidos de forma irregular devem ser devolvidos ao gov

## 5. Executa as chamadas ao Llama e GPT para toda a base de dados

As chamadas ao Groq estão gratuitas, mas tem limite de tempo. Para não estourar o tempo, vou garantir que cada execução do loop dure pelo menos 10 segundos.

In [7]:
import os
import pickle

# Objeto que guardarão a reescrita dos enunciados por documento
enunciado_llama_por_doc = {}
enunciado_gpt_por_doc = {}

# Verifica se os arquivos já existem. Se já existem, recupera.
if os.path.exists(NOME_ARQUIVO_RESULTADO_LLAMA):
    with open(NOME_ARQUIVO_RESULTADO_LLAMA, 'rb') as f:
        enunciado_llama_por_doc = pickle.load(f)
    print("Resultados do Llama recuperados")

if os.path.exists(NOME_ARQUIVO_RESULTADO_GPT):
    with open(NOME_ARQUIVO_RESULTADO_GPT, 'rb') as f:
        enunciado_gpt_por_doc = pickle.load(f)
    print("Resultados do GPT recuperados")


Resultados do Llama recuperados
Resultados do GPT recuperados


In [8]:
import re
from tqdm import tqdm
import time

def remove_html(html):
    return re.sub("<[^>]*>", "", html).strip()

# Percorre o dataframe de ocumentos e gera as queries
for i, row in tqdm(docs.iterrows(), total=len(docs)):
    # Começa a guardar o tempo
    start_time = time.time()
    
    # Extrai a key e o enunciado
    doc_key = row.KEY
    enunciado = remove_html(row.ENUNCIADO)
    
    # Se já tem o enunciado gerado, passa para o próximo
    if doc_key in enunciado_llama_por_doc.keys() and doc_key in enunciado_gpt_por_doc.keys():
        continue
           
    # Gera versões alternativas do enunciado
    enunciado_llama = completa_llama(enunciado)
    enunciado_gpt = completa_gpt(enunciado)
    
    # Salva no mapa
    enunciado_llama_por_doc[doc_key] = enunciado_llama
    enunciado_gpt_por_doc[doc_key] = enunciado_gpt
    
    # Salva em arquivos pickle
    with open(NOME_ARQUIVO_RESULTADO_LLAMA, 'wb') as f:
        pickle.dump(enunciado_llama_por_doc, f)
    
    with open(NOME_ARQUIVO_RESULTADO_GPT, 'wb') as f:
        pickle.dump(enunciado_gpt_por_doc, f)

    # Mede o tempo transcorrido
    elapsed_time = time.time() - start_time
    # Faz um sleep se for necessário
    if elapsed_time < 10:
        time.sleep(10 - elapsed_time)

100%|█████████████████████████████████████████████████████████████████████████| 16045/16045 [45:12:24<00:00, 10.14s/it]
